# Imports

In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

from sklearn.model_selection import train_test_split 
from imblearn.over_sampling import RandomOverSampler


pd.set_option('display.float_format', lambda x: '%.2f' % x)
RSEED = 42

# Import Data

In [132]:
invoice_train = pd.read_csv('data/train/invoice_train.csv')
client_train = pd.read_csv('data/train/client_train.csv')


/var/folders/9x/vhdwn1fd0cldzwf9fnftwf980000gn/T/ipykernel_38057/3220789026.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  invoice_train = pd.read_csv('data/train/invoice_train.csv')


In [133]:
client_train.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.00
1,69,train_Client_1,11,107,29/05/2002,0.00
2,62,train_Client_10,11,301,13/03/1986,0.00
3,69,train_Client_100,11,105,11/07/1996,0.00
4,62,train_Client_1000,11,303,14/10/2014,0.00


In [134]:
invoice_train.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC


## EDA


### Drop Outliers

In [135]:
# Drop outliers high month
invoice_train= invoice_train[invoice_train['months_number']<=36]

### Calculate over Columns

In [136]:
# encode counter_type
d={"ELEC":0,"GAZ":1}
invoice_train['counter_type']=invoice_train['counter_type'].map(d)

## Merge tables

In [137]:
merged_df = client_train.merge(invoice_train, how= "left", on='client_id')
merged_df.head()

,disrict,client_id,client_catg,region,creation_date,target,invoice_date,tarif_type,counter_number,counter_statue,...,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,60,train_Client_0,11,101,31/12/1994,0.00,2014-03-24,11.00,1335667.00,0,...,8.00,1.00,82.00,0.00,0.00,0.00,14302.00,14384.00,4.00,0.00
1,60,train_Client_0,11,101,31/12/1994,0.00,2013-03-29,11.00,1335667.00,0,...,6.00,1.00,1200.00,184.00,0.00,0.00,12294.00,13678.00,4.00,0.00
2,60,train_Client_0,11,101,31/12/1994,0.00,2015-03-23,11.00,1335667.00,0,...,8.00,1.00,123.00,0.00,0.00,0.00,14624.00,14747.00,4.00,0.00
3,60,train_Client_0,11,101,31/12/1994,0.00,2015-07-13,11.00,1335667.00,0,...,8.00,1.00,102.00,0.00,0.00,0.00,14747.00,14849.00,4.00,0.00
4,60,train_Client_0,11,101,31/12/1994,0.00,2016-11-17,11.00,1335667.00,0,...,9.00,1.00,572.00,0.00,0.00,0.00,15066.00,15638.00,12.00,0.00


## Train-Test-Split

In [138]:
y= merged_df['target']
X = merged_df.drop('target', axis = 1)

In [139]:
# test train split only with client-data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RSEED, stratify=y)

In [140]:
X_train.head()

,disrict,client_id,client_catg,region,creation_date,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
1090543,63,train_Client_129653,11,311,31/03/2008,2013-11-02,11.00,1203752.00,0,203.00,6.00,1.00,1200.00,10.00,0.00,0.00,17346.00,18556.00,4.00,0.00
923866,62,train_Client_125100,11,303,03/10/1991,2007-01-18,40.00,29069.00,0,5.00,6.00,1.00,470.00,0.00,0.00,0.00,13262.00,13732.00,4.00,1.00
3577751,62,train_Client_75652,11,307,25/09/2002,2009-08-06,11.00,646915.00,0,203.00,6.00,1.00,460.00,0.00,0.00,0.00,14329.00,14789.00,4.00,0.00
617377,69,train_Client_116688,11,103,05/12/2006,2007-01-17,11.00,1067929.00,1,207.00,6.00,1.00,2.00,0.00,0.00,0.00,0.00,2.00,2.00,0.00
3117610,60,train_Client_63112,11,101,12/12/1978,2019-01-31,11.00,299341.00,0,410.00,9.00,1.00,2400.00,176.00,0.00,0.00,14223.00,16799.00,12.00,0.00


### Calculate Over Rows

In [141]:
# Transactions Counts
transaction_counts_df = X_train.groupby('client_id').size().reset_index(name='transactions_count'.format('1'))
transaction_counts_df
X_train = X_train.merge(transaction_counts_df, how='left', on='client_id' )
X_train.sort_values('client_id').head()

,disrict,client_id,client_catg,region,creation_date,invoice_date,tarif_type,counter_number,counter_statue,counter_code,...,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,transactions_count
2338913,60,train_Client_0,11,101,31/12/1994,2013-07-22,11.00,1335667.00,0,203.00,...,1.00,147.00,0.00,0.00,0.00,13983.00,14130.00,4.00,0.00,24
202079,60,train_Client_0,11,101,31/12/1994,2009-07-27,11.00,1335667.00,0,203.00,...,1.00,312.00,0.00,0.00,0.00,5906.00,6218.00,4.00,0.00,24
1728348,60,train_Client_0,11,101,31/12/1994,2008-01-04,11.00,1335667.00,0,203.00,...,1.00,277.00,0.00,0.00,0.00,4969.00,5246.00,4.00,0.00,24
369318,60,train_Client_0,11,101,31/12/1994,2013-03-29,11.00,1335667.00,0,203.00,...,1.00,1200.00,184.00,0.00,0.00,12294.00,13678.00,4.00,0.00,24
535189,60,train_Client_0,11,101,31/12/1994,2012-11-29,11.00,1335667.00,0,203.00,...,1.00,886.00,0.00,0.00,0.00,11408.00,12294.00,4.00,0.00,24


In [ ]:
# Cosummation per Time  

# invoice_train['invoice_date'] = pd.to_datetime(invoice_train['invoice_date'])
# # Calculate the Consumption Amount:
# invoice_train['consommation_amount'] = invoice_train['consommation_level_1'] + invoice_train['consommation_level_2'] + invoice_train['consommation_level_3'] + invoice_train['consommation_level_4']
# # Calculate the Time Period Between Invoices:
# invoice_train = invoice_train.sort_values(by=['client_id', 'counter_type', 'invoice_date'])
# invoice_train['invoice_period'] = invoice_train.groupby(['client_id', 'counter_type'])['invoice_date'].diff()



In [ ]:
# most frequent reading_remarque (Maria)

In [ ]:
# mean counter_coefficient (Maria)

In [ ]:
# mean month_numbers by client (Jesus)
# 1. for each client: calculate mean month number
# 2. create a new column "month_number_mean"


Columns which we can't aggregate to one single client:
- invoice_date
- tarif
- counter_number
- counter_statue
- counter_code

In [ ]:
# Check how many values, if we could aggregate specific features to one single client
# grouped = invoice_train.groupby('client_id')
# test = grouped['counter_type'].nunique()
# test.unique()
# counter_counts_df = invoice_train.groupby('client_id').size().reset_index(name='transactions_count'.format('1'))


In [ ]:
# drop columns we dont aggregate

In [ ]:
# delete duplicates

### Random Oversampler


In [ ]:
# handling the imbalanced
ros = RandomOverSampler(random_state=RSEED)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
sns.countplot(x=y_train)